In [4]:
%load_ext autoreload
%autoreload 2
import pickle
import numpy as np
from collections import namedtuple, defaultdict
from paper_sims_util import MCC
from running_wrappers import get_loaders
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
LOADERS = get_loaders()
AlgoParams = namedtuple('AlgoParams', 'stability_samples M pi')
GraphParams = namedtuple('GraphParams', 'N eta p d ratios')
import pandas as pd
import time

In [5]:
from final_algo import tiger_algo, glasso_vanilla, new_algo
from paper_sims_util import MCC, grid_graph, grid_3D, chain, star, random_graph, is_MTP2
import pickle
import random

In [13]:
graph_params_dict = {
    'chain': GraphParams(p=100, N=[25, 50, 100, 200, 500, 1000], eta=1, ratios=None, d=None), #p, N, eta
    'star': GraphParams(p=100, d=[10, 20, 30, 50], N=50, eta=1, ratios=None), #p, d, N, eta
    'random': GraphParams(p=100, d=0.01, ratios=[r/100. for r in [25, 50, 100, 200, 500, 1000]], eta=1, N=None), #p, d, ratio over 500, eta
    #'grid_3D': GraphParams(p=4, ratios=new_grid_ratios, eta=2, N=None, d=None), #p, ratio over 524, eta
    'grid': GraphParams(p=10, ratios=[r/100. for r in [25, 50, 100, 200, 500,1000]], eta=1, N=None, d=None) #p, ratio over 529, eta
}

In [7]:
algo_params = AlgoParams(stability_samples=50, M=7./9., pi=0.8)
graph_params = graph_params_dict['random']

In [9]:
run_name = 'NIPS_rebuttal_ALL'
run_id = 1

In [163]:
from running_wrappers import random_wrapper
random_wrapper(graph_params, algo_params, run_name, run_id)

Warning using rebuttal ALGOS
Currently on our
Running new algorithm
N=25, M=12
Working on l = 0
Working on l = 1
Working on l = 2
Algorithm our got 0.4616937218441082 MCC
Currently on tiger
b'[1] "Loaded X"\n'
b'High-deimensional Sparse Undirected Graphical Models.\n'
b'Tuning-Insensitive Graph Estimation and Regression.\n'
b'[1] "DONE with tiger"\n'
b''
Algorithm tiger got 0 MCC
Currently on clime
b'[1] "Loaded X"\n'
b'High-deimensional Sparse Undirected Graphical Models.\n'
b'The Constrained L1 Minimization for Sparse Precision Matrix Estimation.\n'
b'[1] "DONE with clime"\n'
b''
Algorithm clime got 0 MCC
Warning using rebuttal ALGOS
Currently on our
Running new algorithm
N=50, M=20
Working on l = 0
Working on l = 1
Working on l = 2
Working on l = 3
Algorithm our got 0.5352685946455787 MCC
Currently on tiger
b'[1] "Loaded X"\n'
b'High-deimensional Sparse Undirected Graphical Models.\n'
b'Tuning-Insensitive Graph Estimation and Regression.\n'
b'[1] "DONE with tiger"\n'
b''
Algorithm t

In [161]:
from running_wrappers import chain_wrapper
chain_wrapper(graph_params_dict['chain'], algo_params, run_name, run_id)

Warning using rebuttal ALGOS
Currently on our
Running new algorithm
N=25, M=12
Working on l = 0
Working on l = 1
Working on l = 2
Working on l = 3
Algorithm our got 0.6076637164360463 MCC
Currently on tiger
b'[1] "Loaded X"\n'
b'High-deimensional Sparse Undirected Graphical Models.\n'
b'Tuning-Insensitive Graph Estimation and Regression.\n'
b'[1] "DONE with tiger"\n'
b''
Algorithm tiger got 0 MCC
Currently on clime
b'[1] "Loaded X"\n'
b'High-deimensional Sparse Undirected Graphical Models.\n'
b'The Constrained L1 Minimization for Sparse Precision Matrix Estimation.\n'
b'[1] "DONE with clime"\n'
b''
Algorithm clime got -0.014136305599865527 MCC
Warning using rebuttal ALGOS
Currently on our
Running new algorithm
N=50, M=20
Working on l = 0
Working on l = 1
Working on l = 2
Working on l = 3
Algorithm our got 0.9364717554170788 MCC
Currently on tiger
b'[1] "Loaded X"\n'
b'High-deimensional Sparse Undirected Graphical Models.\n'
b'Tuning-Insensitive Graph Estimation and Regression.\n'
b'[1]

In [16]:
from running_wrappers import grid_wrapper
grid_wrapper(graph_params_dict['grid'], algo_params, run_name, 2)

Warning using rebuttal ALGOS
Currently on tiger
b'[1] "Loaded X"\n'
b'High-deimensional Sparse Undirected Graphical Models.\n'
b'Tuning-Insensitive Graph Estimation and Regression.\n'
b'[1] "DONE with tiger"\n'
b''
Algorithm tiger got 0 MCC
Warning using rebuttal ALGOS
Currently on tiger
b'[1] "Loaded X"\n'
b'High-deimensional Sparse Undirected Graphical Models.\n'
b'Tuning-Insensitive Graph Estimation and Regression.\n'
b'[1] "DONE with tiger"\n'
b''
Algorithm tiger got 0 MCC
Warning using rebuttal ALGOS
Currently on tiger
b'[1] "Loaded X"\n'
b'High-deimensional Sparse Undirected Graphical Models.\n'
b'Tuning-Insensitive Graph Estimation and Regression.\n'
b'[1] "DONE with tiger"\n'
b''
Algorithm tiger got 0 MCC
Warning using rebuttal ALGOS
Currently on tiger
b'[1] "Loaded X"\n'
b'High-deimensional Sparse Undirected Graphical Models.\n'
b'Tuning-Insensitive Graph Estimation and Regression.\n'
b'[1] "DONE with tiger"\n'
b''
Algorithm tiger got 0.5653925887299175 MCC
Warning using rebut

In [143]:
def get_results_tiger(graph_params, algo_params, omega, N):
    algos = {'tiger': tiger_algo, 'our': new_algo}
    is_MTP2(omega)
    sigma = np.linalg.inv(omega)
    np.random.seed(random.SystemRandom().randint(0, 2**32-2))
    X = np.random.multivariate_normal(mean = np.zeros(omega.shape[0]), 
                                      cov = np.linalg.inv(omega), 
                                      size = N)
    
    ALL_RESULTS = {}
    for algo_name, algo in algos.items():
        print("Currently on {}".format(algo_name))
        if 'glasso' in algo_name:
            results = algo(X, 0.0001)
        else:
            results = algo(X)
        
        omega_hat = results #only true for some of these
        ALL_RESULTS[algo_name] = results
        res_mcc = MCC(omega_hat, omega)
        print("Algorithm {} got {} MCC".format(algo_name, res_mcc))
    return ALL_RESULTS

In [146]:
def random_wrapper(graph_params, algo_params, run_name, run_id):
    p = graph_params.p
    d = graph_params.d
    dim = p
    Ns = [int(r*dim) for r in graph_params.ratios]
    if max(Ns) < 10:
        Ns = [x*10 for x in Ns]
    for r, N in zip(graph_params.ratios, Ns):
        omega = random_graph(p,d)
        inv_exists = False
        while not inv_exists:
            try:
                sigma = np.linalg.inv(omega)
                inv_exists = True
            except:
                omega = random_graph(p,d)
        results = get_results_tiger(graph_params, algo_params, omega, N)
        fname = "{}_{}_{}_{}_results.pkl".format(run_name, run_id, 'random', r)
        with open(fname, 'wb') as f:
            pickle_obj = (omega, results)
            pickle.dump(pickle_obj, f)

In [147]:
random_wrapper(graph_params, algo_params, run_name, run_id)

Currently on tiger
b'[1] "Loaded X"\n'
b'High-deimensional Sparse Undirected Graphical Models.\n'
b'Tuning-Insensitive Graph Estimation and Regression.\n'
b'[1] "DONE with tiger"\n'
b''
Algorithm tiger got 0 MCC
Currently on our
Running new algorithm
N=25, M=15
Working on l = 0
Working on l = 1
Working on l = 2
Working on l = 3
Algorithm our got 0.3091784075501798 MCC
Currently on tiger
b'[1] "Loaded X"\n'
b'High-deimensional Sparse Undirected Graphical Models.\n'
b'Tuning-Insensitive Graph Estimation and Regression.\n'
b'[1] "DONE with tiger"\n'
b''
Algorithm tiger got 0 MCC
Currently on our
Running new algorithm
N=50, M=27
Working on l = 0
Working on l = 1
Working on l = 2
Working on l = 3
Working on l = 4
Working on l = 5
Algorithm our got 0.5905562029438363 MCC
Currently on tiger
b'[1] "Loaded X"\n'
b'High-deimensional Sparse Undirected Graphical Models.\n'
b'Tuning-Insensitive Graph Estimation and Regression.\n'
b'[1] "DONE with tiger"\n'
b''
Algorithm tiger got 0.3763552910572519